<a href="https://colab.research.google.com/github/super-onigiri-man/ITE3-2/blob/main/FM_%E3%83%99%E3%82%B9%E3%83%88%E3%83%92%E3%83%83%E3%83%88%E3%83%A9%E3%83%B3%E3%82%AD%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import requests
from bs4 import BeautifulSoup
import datetime
import pandas as pd

def selecter():

    print("見たい週を選択してください")
    select = int(input("1:今回 2:前回 3:別の回"))
    return select

def OriconTodays():
    # 今日の日付を求める
    d_today = datetime.date.today()
    # 今日の曜日を求める
    todayweek = datetime.date.today().weekday()
    #print(d_today, todayweek)

    #オリコンの発表は毎週水曜日のため、火曜日までは先週のランキングを表示
    #日付は来週の水曜日付となる。
    if (todayweek == 0):  # 今日が月曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today
    elif (todayweek == 1):# 今日が火曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today - datetime.timedelta(days=1)
    elif (todayweek == 2):# 今日が水曜日
        Oriconday = d_today + datetime.timedelta(days=5)
    elif (todayweek == 3):# 今日が木曜日
        Oriconday = d_today + datetime.timedelta(days=4)
    elif (todayweek == 4):# 今日が金曜日
        Oriconday = d_today + datetime.timedelta(days=3)
    elif (todayweek == 5):# 今日が土曜日
        Oriconday = d_today + datetime.timedelta(days=2)
    elif (todayweek == 6):# 今日が日曜日
        Oriconday = d_today + datetime.timedelta(days=1)

    print(Oriconday.year,Oriconday.month,Oriconday.day)

    return Oriconday


def OriconLastWeek():
    # 今日の日付を求める
    d_today = datetime.date.today()
    # 今日の曜日を求める
    todayweek = datetime.date.today().weekday()
    # print(d_today, todayweek)

    # オリコンの発表は毎週水曜日のため、火曜日までは先週のランキングを表示
    # 日付は来週の月曜日付となる。
    if (todayweek == 0):  # 今日が月曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today
    elif (todayweek == 1):  # 今日が火曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today - datetime.timedelta(days=1)
    elif (todayweek == 2):  # 今日が水曜日
        Oriconday = d_today - datetime.timedelta(days=2)
    elif (todayweek == 3):  # 今日が木曜日
        Oriconday = d_today - datetime.timedelta(days=3)
    elif (todayweek == 4):  # 今日が金曜日
        Oriconday = d_today - datetime.timedelta(days=4)
    elif (todayweek == 5):  # 今日が土曜日
        Oriconday = d_today - datetime.timedelta(days=5)
    elif (todayweek == 6):  # 今日が日曜日
        Oriconday = d_today - datetime.timedelta(days=6)

    return Oriconday


def OriconSelectWeek():
    # 日付の入力を促す
    date = input("2020年8月3日以降の日付を入力してください (YYYY-MM-DD): ")

    # 入力された日付をdatetimeオブジェクトに変換
    dt = datetime.datetime.strptime(date, "%Y-%m-%d")
    dt = dt.date()
    # 曜日を取得
    weekday = dt.weekday()

    # オリコンの発表は毎週水曜日のため、火曜日までは先週のランキングを表示
    # 日付は来週の月曜日付となる。
    if (weekday == 0):  # 今日が月曜日
        Oriconday = dt
    elif (weekday == 1):  # 今日が火曜日
        Oriconday = dt - datetime.timedelta(days=1)
    elif (weekday == 2):  # 今日が水曜日
        Oriconday = dt - datetime.timedelta(days=2)
    elif (weekday == 3):  # 今日が木曜日
        Oriconday = dt - datetime.timedelta(days=3)
    elif (weekday == 4):  # 今日が金曜日
        Oriconday = dt - datetime.timedelta(days=4)
    elif (weekday == 5):  # 今日が土曜日
        Oriconday = dt - datetime.timedelta(days=5)
    elif (weekday == 6):  # 今日が日曜日
        Oriconday = dt - datetime.timedelta(days=6)

    return Oriconday


def OriconWeekRank(Oriconday):#オリコン週間ランキング

    #1位から10位
    load_url = "https://www.oricon.co.jp/rank/js/w/" + str(Oriconday) + "/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2',class_='title') #曲名
    artist = soup.find(class_="content-rank-main").find_all('p',class_='name') #アーティスト名
    score = 6.0 #独自スコア
    rank = 1 #ランキング
    print(str(Oriconday) + "付けオリコン週間シングルランキング")

    for link, artist in zip(links, artist):

        if rank != 10:#10位以下（１ケタの場合）なら（点数の位置を揃えるため）
            print("第" + str(rank) + "位 " + " {:.1f}　 ".format(score) + link.text + "/" + artist.text)
        else: #10位（２ケタの場合）なら
            print("第" + str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
        rank = rank + 1
        score = score - 0.3


    #11位から20位
    load_url = "https://www.oricon.co.jp/rank/js/w/" + str(Oriconday) + "/p/2/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2', class_='title')  # 曲名
    artist = soup.find(class_="content-rank-main").find_all('p', class_='name')  # アーティスト名
    for link, artist in zip(links, artist):
         print("第" + str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
         rank = rank + 1
         score = score - 0.3



def OriconDigitalRank(Oriconday):#オリコンデジタルシングルランキング
    # 1位から10位
    load_url = "https://www.oricon.co.jp/rank/dis/w/" + str(Oriconday) + "/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2', class_='title')
    artist = soup.find(class_="content-rank-main").find_all('p', class_='name')  # アーティスト名
    rank = 1
    score = float(6.0)
    print(str(Oriconday) + "付けオリコン週間デジタルシングルランキング")
    for link, artist in zip(links, artist):
        if rank != 10:
            print("第" + str(rank) + "位 " + " {:.1f}　 ".format(score) + link.text + "/" + artist.text)
        else:
            print("第" + str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
        rank = rank + 1
        score = score - 0.3

    # 11位から20位
    load_url = "https://www.oricon.co.jp/rank/dis/w/" + str(Oriconday) + "/p/2/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2', class_='title')
    artist = soup.find(class_="content-rank-main").find_all('p', class_='name')  # アーティスト名
    for link, artist in zip(links, artist):
        print("第" + str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
        rank = rank + 1
        score = score - 0.3





sel = int(selecter())

if sel==1: #今週のランキング表示
    OriconTodays()
    OriconWeekRank(OriconTodays())
    print()
    OriconDigitalRank(OriconTodays())

elif sel == 2: #前週のランキング表示
    OriconLastWeek()
    OriconWeekRank(OriconLastWeek())
    print()
    OriconDigitalRank(OriconLastWeek())

else : #別の週のランキング
    OriconWeekRank(OriconSelectWeek())
    print()
    OriconDigitalRank(OriconSelectWeek())

見たい週を選択してください
1:今回 2:前回 3:別の回1
2023 11 6
2023 11 6
2023-11-06付けオリコン週間シングルランキング
第1位  6.0　 絶体絶命/Beautiful/AS ONE/ジャニーズWEST
第2位  5.7　 すっごいFEVER!/Wake-up Call〜目覚めるとき〜/Neverending Shine/モーニング娘。’23
第3位  5.4　 承認欲求/櫻坂46
第4位  5.1　 アイドルなんかじゃなかったら/AKB48
第5位  4.8　 つまり、ぞっこんLOVE/CUBERS
第6位  4.5　 No more cry/エレファントカシマシ
第7位  4.2　 ユウマガドキ/そらるとりぶ
第8位  3.9　 夢で逢えなくても/指先ハート/大西亜玖璃
第9位  3.6　 ひつじがいっぴき/下剋上/Vivid BAD SQUAD
第10位 3.3　 初心LOVE(うぶらぶ)/なにわ男子
第11位 3.0　 ひかりのあと/skip/上白石萌音
第12位 2.7　 THE IDOLM@STER MILLION ANIMATION THE@TER MILLIONSTARS Team8th『REFRAIN REL@TION』/MILLIONSTARS Team8th
第13位 2.4　 THE IDOLM@STER MILLION ANIMATION THE@TER MILLIONSTARS Team3rd『オレンジノキオク』/MILLIONSTARS Team3rd
第14位 2.1　 THE IDOLM@STER MILLION ANIMATION THE@TER MILLIONSTARS Team7th『トワラー』/MILLIONSTARS Team7th
第15位 1.8　 THE IDOLM@STER MILLION ANIMATION THE@TER MILLIONSTARS Team5th『バトンタッチ』/MILLIONSTARS Team5th
第16位 1.5　 THE IDOLM@STER MILLION ANIMATION THE@TER MILLIONSTARS Team6th『Unknown Boxの開き方』/MILLIONSTARS Team6th
第17位 1.2　 想わせぶりっこ/≠

In [11]:

def OricondayTodays():
    # 今日の日付を求める
    d_today = datetime.date.today()
    # 今日の曜日を求める
    todayweek = datetime.date.today().weekday()
    #print(d_today, todayweek)

       # オリコンの発表は毎週水曜日のため、火曜日までは先週のランキングを表示
    # 日付は来週の月曜日付となる。
    if (todayweek == 0):  # 今日が月曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today
    elif (todayweek == 1):  # 今日が火曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today - datetime.timedelta(days=1)
    elif (todayweek == 2):  # 今日が水曜日
        Oriconday = d_today - datetime.timedelta(days=2)
    elif (todayweek == 3):  # 今日が木曜日
        Oriconday = d_today - datetime.timedelta(days=3)
    elif (todayweek == 4):  # 今日が金曜日
        Oriconday = d_today - datetime.timedelta(days=4)
    elif (todayweek == 5):  # 今日が土曜日
        Oriconday = d_today - datetime.timedelta(days=5)
    elif (todayweek == 6):  # 今日が日曜日
        Oriconday = d_today - datetime.timedelta(days=6)


    return Oriconday

def BillboadRank(Oriconday):#ビルボードJAPAN HOT100ランキング
    # 1位から10位
    load_url = "https://www.billboard-japan.com/charts/detail?a=hot100&year="+str(Oriconday.year)+"&month="+str(Oriconday.month)+"&day="+str(Oriconday.day)
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="rank1").find_all('p', class_='musuc_title')
    artist = soup.find(class_="rank1").find_all('p', class_='artist_name')  # アーティスト名
    rank = 1
    score = float(6.0)
    print(str(Oriconday) + "付けオリコン週間デジタルシングルランキング")
    for link, artist in zip(links, artist):
      s = link.text+"/"+artist.text
      result = ''.join(s.split())

      if rank != 10:
            print("第" + str(rank) + "位 " + " {:.1f}　 ".format(score) + result)
      else:
            print("第" + str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
      rank = rank + 1
      score = score - 0.3

    if rank == 20:
      exit()

OricondayTodays()
BillboadRank(OricondayTodays())


2023-10-30付けオリコン週間デジタルシングルランキング
第1位  6.0　 承認欲求/櫻坂46
